# Regras de Associação

***


## Índice

1. [Motivação](#motivação)
2. [Apriori](#apriori)
3. [FP-Growth](#fp-growth)

## Motivação

Digamos que uma rede multinacional de supermercados precisa de você para saber quais conjuntos de itens são comprados com maior frequência. Essa informação será usada para criar um combo promocional com esses itens, que deverá aumentar a receita da companhia no período de Natal desse ano. Essa campanha é muito importante, pois a multinacional irá para IPO no ano que vem e um número sólido de vendas nesse período é extremamente importante.

Seu chefe lhe pediu para extrair as combinações mais frequentes, e disponibilizou um dataset que possui uma fração das vendas dos todos os produtos da empresa. Cada linha do dataset representa a compra de um item. Se a pessoa fez uma compra com vários itens diferentes, cada item será discriminado em uma linha nova. Entretanto, a quantidade de cada item tem uma coluna específica.

Os datasets tratados até agora consistiam em um conjunto de instâncias que possuem um conjunto de features $ X = \{x_i | i \geq 1 \wedge x_i = \{f_i \in \mathbb{R} | i >= 1\}\} $. Entretanto, regras de associação são algoritmos que extraem conjuntos de itens frequentes, em datasets que cada instância é um conjunto de itens. Vamos visualizar o que isso significa.

Vamos observar o dataset de compras de um supermercado, esse dataset não está disponível no _sklearn_. Portanto, precisaremos baixa-lo utilizando o comando _curl_. E carregá-lo utilizando pandas.


In [ ]:
# baixando o dataset

!curl -o ./online_retail.xlsx http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx

import pandas as pd
import numpy as np

df = pd.read_excel("./online_retail.xlsx")

Vamos visualizar uma pequena parte desse dataset.

In [ ]:
df.head()

Aqui temos diversas informações sobre compras em um supermercado:

- (_InvoiceNo_) o número da fatura, identificador de uma compra
- (_StockCode_) o código de certo produto no estoque
- (_Description_) a descrição do produto
- (_Quantity_) a quantidade de produtos que foi comprada
- (_InvoiceDate_) o dia da compra
- (_UnitPrice_) o preço por unidade
- (_CustomerID_) o id do consumidor
- (_Country_) o país de venda

In [ ]:
# vamos olhar todos os países existentes

print(df["Country"].unique())

# dos países, vamos escolher apenas as vendas feitas no Reino Unido, apenas pelo fato de existirem mais vendas no dataset
country = "United Kingdom"
sales = df[df["Country"] == country]

In [ ]:
# antes de utilizar nosso dataset, precisamos fazer alguns pre-processamentos:
# remover todas as instãncias que não possuem _InvoiceNo_
filtered_sales = sales.dropna(axis=0, subset=["InvoiceNo"])
# transformar em strings
filtered_sales["InvoiceNo"] = filtered_sales["InvoiceNo"].astype("str")
# remover instancias que contenham "C" no _InvoiceNo_, representando instancias que não possuem essa feature
filtered_sales = filtered_sales[~filtered_sales["InvoiceNo"].str.contains("C")]

# remover espaços desnecessários no começo e fim de cada _Description_
filtered_sales["Description"] = filtered_sales["Description"].str.strip()
filtered_sales["Description"] = filtered_sales["Description"].astype("str")


Como o dataset a ser avaliado deverá consistir em conjuntos de itens, vamos transformar cada compra em um conjunto de itens. Note que um conjunto não possui informação sobre quantidade de elementos repetidos. Portanto, a coluna com a contagem de cada item deve ser removida, indicando apenas a presença daquele item em uma compra.

In [ ]:
sales_set = filtered_sales.groupby(['InvoiceNo', 'Description'])["Quantity"].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
sales_set.head()

In [ ]:
# existem algumas vendas que tiveram algum tipo de problema, vamos remove-las
sales_set = sales_set.iloc[:, :4057]

# também estão descritos o tipo de postagem, se pela internet ou não, vamos remove-los
sales_set = sales_set.drop("POSTAGE", axis=1)
sales_set = sales_set.drop("DOTCOM POSTAGE", axis=1)

# nosso dataset final
sales_set.head()

In [ ]:
# agora, vamos transformar o dataset de uma contagem de itens, para apenas conjuntos
# faremos isso transformando toda contagem para uma presença ou não do item

count_to_set = lambda x: x > 0 # aqui se uma venda possui pelo menos 1 item, afime True, caso contrário, False
sales_set = sales_set.applymap(count_to_set)
sales_set.head()

Após esse paço de pre-processamento, conseguimos extrair quais itens são comprados em conjunto com maior frequência para todas as vendas.

## Apriori

Inicialmente, vamos utilizar o algoritmo a priori. Nele precisamos indicar qual a repetição mínima necessária de repetição que buscamos. Por exemplo, se quisermos apenas as repetições que aconteçam pelo menos $n\%$ das vezes.

O algoritmo funciona por criar todas as combinações possíveis de conjuntos e então checar suas frequências.

In [ ]:
from mlxtend.frequent_patterns import apriori

# use_colnames retorna o itemset como nomes ao invés de indices das colunas
frequency_set = apriori(sales_set, min_support=0.01, use_colnames=True)
print(frequency_set)

In [ ]:
# perceba que temos muitas conjuntos com apenas um elemento
# vamos filtrar esses conjuntos que tem apenas um elemento

def filter_set_lenght(input_set, lenght=2):
    input_set['set_lenght'] = input_set['itemsets'].apply(lambda x: len(x))
    new_set = input_set[input_set['set_lenght'] >= lenght]
    new_set.reset_index(inplace=True, drop=True)
    return new_set
    
combo_set = filter_set_lenght(frequency_set)
print(combo_set)

In [ ]:
# escolhendo a combinação de itens com maior repetição
def get_highest_support(input_set):
    instance = input_set.iloc[input_set["support"].idxmax()]
    return instance

# e criando uma função auxiliar para mostrar as estatísticas do nosso combo a partir do nosso conjunto de vendas
def print_combo_stats(combo, sales):
    print("itens: %s"%(str(tuple(combo["itemsets"]))))
    print("frequencia %.2f%%" %(float(combo["support"])*100))
    print("vendas totais do combo: %d"%(float(combo["support"]) * sales.shape[0]))

combo = get_highest_support(combo_set)
print_combo_stats(combo, sales_set)

## FP-Growth

Ao contrário do algoritmo a priori, FP-Growth não precisa criar todas os conjuntos de combinações. Para datasets em que a quantidade de combinações é muito grande, ele possui uma vantagem sobre seu tempo de execução.

In [ ]:
# caso o import fpgrowth falhe, descomente a linha abaixo
# !pip install mlxtend -U

from mlxtend.frequent_patterns.fpgrowth import fpgrowth

# use_colnames retorna o itemset como nomes ao invés de indices das colunas
frequency_set = fpgrowth(sales_set, min_support=0.01, use_colnames=True)
print(frequency_set)

In [ ]:
combo_set = filter_set_lenght(frequency_set)
combo = get_highest_support(combo_set)
print_combo_stats(combo, sales_set)

Temos o mesmo resultado, mas vamos avaliar o tempo de execução entre cada algoritmo...

In [ ]:
from time import time
t0 = time()
frequency_set = apriori(sales_set, min_support=0.01, use_colnames=True)
t1 = time()
frequency_set = fpgrowth(sales_set, min_support=0.01, use_colnames=True)
t2 = time()

print("APriori(t_delta): %f" %(t1-t0))
print("FP-Growth(t_delta): %f" %(t2-t1))